# 학교 안전사고 현황 분석

- 코드 작성자 : 정하윤
- 코드 작성일 : 2024-07-06

맡은 부분 알아서 잘 정리해주세요!!🥰

## 0. 라이브러리 및 데이터 임포트

### 라이브러리 설정

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd

import matplotlib.pyplot as plt
import folium
import json

### 데이터 불러오기

In [ ]:
# 엑셀 파일 경로
file_path = r'C:\Users\user\Desktop\학교안전사고 데이터분석 공모전\하윤\data\중앙회 제공 데이터_수정\★2019~2023 학교안전사고 데이터_수정.xlsx'

# 파일 불러오기
xlsfile = pd.ExcelFile(file_path)

### 데이터 전처리

- 기존 데이터의 연도별 정렬 → 사고 접수 일자 기준으로 정렬
- 사고 발생 일자 기준으로 데이터 생성
- 연도별 데이터 프레임 생성
- 2023년 데이터에서 '사고매개물' 컬럼이 '매개물'로 표기되어 있어 변경 후 통합 데이터 생성

In [ ]:
# 모든 시트 이름 출력
print(xlsfile.sheet_names)

['데이터 설명', '(참고) 사고유형별 항목', '2019', '2020', '2021', '2022', '2023']


In [ ]:
# 처리할 시트 이름 리스트
sheet_names = ['2019', '2020', '2021', '2022', '2023']

In [ ]:
# 통합 데이터프레임 생성
dataframes = []
for sheet_name in sheet_names:
    df = xlsfile.parse(sheet_name)
    # '매개물' 컬럼명을 '사고매개물'로 변경
    if '매개물' in df.columns:
        df.rename(columns={'매개물': '사고매개물'}, inplace=True)
    dataframes.append(df)
combined_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
# '사고발생일' 컬럼을 datetime 형식으로 변환
combined_df['사고발생일'] = pd.to_datetime(combined_df['사고발생일'], errors='coerce')

# 2019년 이전의 데이터 제외
filtered_df = combined_df[combined_df['사고발생일'].dt.year >= 2019]

# 날짜순으로 정렬한 뒤 인덱스 초기화
filtered_df = filtered_df.sort_values(by='사고발생일').reset_index(drop=True)

In [ ]:
# 연도별 데이터프레임 생성
yearly_data = {year: filtered_df[filtered_df['사고발생일'].dt.year == year] for year in filtered_df['사고발생일'].dt.year.unique()}

In [ ]:
# 연도별 데이터 프레임 생성
df2019 = yearly_data[2019].reset_index(drop=True)
df2020 = yearly_data[2020].reset_index(drop=True)
df2021 = yearly_data[2021].reset_index(drop=True)
df2022 = yearly_data[2022].reset_index(drop=True)
df2023 = yearly_data[2023].reset_index(drop=True)

### 데이터 프레임 확인

In [ ]:
df2019.head()

,구분,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생일,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물
0,A0000209,중학교,경기,안양과천교육지원청,공립,일반학생,남,1학년,2019-01-02,수,09:00,수업시간,부속시설,다리,낙상-미끄러짐,기타운동,건물(문/창문/바닥/벽 등)
1,A0001299,고등학교,경기,경기도교육청,공립,일반학생,여,1학년,2019-01-02,수,09:00,수업시간,교실,머리(두부),사람과의 충돌,장난/놀이,자연(사람/동물/식물 등)
2,A0001300,초등학교,경기,구리남양주교육지원청,공립,일반학생,남,3학년,2019-01-02,수,13:48,점심시간,교실,손,물리적힘 노출,식사/수면/휴식,가구(책상/의자/책장/탁자/침대 등)
3,A0001306,중학교,경기,성남교육지원청,공립,일반학생,여,1학년,2019-01-02,수,12:30,수업시간,통로,발,낙상-넘어짐,보행/주행,건물(문/창문/바닥/벽 등)
4,A0001310,중학교,경기,이천교육지원청,공립,일반학생,남,2학년,2019-01-02,수,12:25,수업시간,교실,치아(구강),물리적힘 노출,기타,자연(사람/동물/식물 등)


In [ ]:
df2020.head()

,구분,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생일,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물
0,B0006908,중학교,전남,고흥교육지원청,공립,일반학생,남,1학년,2020-01-01,수,09:30,학교행사,운동장,발,낙상-떨어짐,보행/주행,기타
1,C0057303,고등학교,충북,충청북도교육청,공립,체육특기학생,남,1학년,2020-01-01,수,00:00,특별활동,부속시설,다리,물리적힘 노출,기타운동,자연(사람/동물/식물 등)
2,B0001475,초등학교,인천,남부교육지원청,공립,일반학생,남,3학년,2020-01-02,목,12:09,수업시간,운동장,치아(구강),낙상-넘어짐,보행/주행,기타
3,B0002373,초등학교,경기,고양교육지원청,공립,일반학생,남,1학년,2020-01-02,목,09:00,등하교,통로,치아(구강),낙상-넘어짐,보행/주행,건물(문/창문/바닥/벽 등)
4,B0002237,고등학교,대구,대구교육청,사립,일반학생,남,2학년,2020-01-02,목,09:35,수업시간,운동장,손,사람과의 충돌,구기운동,자연(사람/동물/식물 등)


In [ ]:
df2021.head()

,구분,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생일,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물
0,C0000178,유치원,충북,옥천교육지원청,사립,일반학생,남,유아,2021-01-01,금,15:30,기타,통로,기타,낙상-넘어짐,기타,건물(문/창문/바닥/벽 등)
1,C0000357,유치원,대구,서부교육지원청,사립,일반학생,여,유아,2021-01-04,월,16:30,등하교,교외활동,복합부위,사람과의 충돌,기타,자연(사람/동물/식물 등)
2,C0000309,초등학교,전북,전주교육지원청,공립,일반학생,남,2학년,2021-01-04,월,10:30,휴식시간 및 청소시간,교실,복합부위,물리적힘 노출,보행/주행,가구(책상/의자/책장/탁자/침대 등)
3,C0000407,중학교,전남,무안교육지원청,공립,체육특기학생,남,3학년,2021-01-04,월,11:30,체육수업,부속시설,치아(구강),물리적힘 노출,기타운동,자연(사람/동물/식물 등)
4,C0000343,중학교,경기,광주하남교육지원청,공립,일반학생,남,1학년,2021-01-04,월,13:40,점심시간,교실,팔,사람과의 충돌,장난/놀이,가구(책상/의자/책장/탁자/침대 등)


In [ ]:
df2022.head()

,구분,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생일,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물
0,D0007141,고등학교,경북,경상북도교육청,사립,일반학생,남,1학년,2022-01-01,토,18:00,기숙사생활,운동장,복합부위,낙상-넘어짐,보행/주행,자연(사람/동물/식물 등)
1,D0075781,고등학교,인천,인천광역시교육청,사립,체육특기학생,남,2학년,2022-01-02,일,16:30,특별활동,운동장,다리,낙상-넘어짐,기타,자연(사람/동물/식물 등)
2,D0001082,중학교,제주,제주시교육지원청,사립,일반학생,여,3학년,2022-01-02,일,10:30,특별활동,교실,기타,기타,기타운동,자연(사람/동물/식물 등)
3,D0001353,중학교,경기,성남교육지원청,공립,일반학생,남,2학년,2022-01-03,월,12:40,점심시간,운동장,발,낙상-넘어짐,구기운동,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등)
4,D0000748,고등학교,경기,고양교육지원청,공립,일반학생,남,2학년,2022-01-03,월,10:30,체육수업,부속시설,발,기타,구기운동,자연(사람/동물/식물 등)


In [ ]:
df2023.head()

,구분,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생일,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물
0,E0001189,고등학교,경기,화성오산교육지원청,사립,체육특기학생,남,2학년,2023-01-01,일,19:30,기타,부속시설,다리,사람과의 충돌,기타운동,자연(사람/동물/식물 등)
1,E0001244,중학교,부산,해운대교육지원청,공립,일반학생,남,2학년,2023-01-02,월,11:00,체육활동,부속시설,머리(두부),사람과의 충돌,구기운동,자연(사람/동물/식물 등)
2,E0001567,고등학교,경기,평택교육지원청,사립,일반학생,여,2학년,2023-01-02,월,12:10,체육활동,부속시설,발,염좌·삐임 등 신체 충격,구기운동,기타
3,E0003543,고등학교,전남,여수교육지원청,사립,일반학생,남,2학년,2023-01-02,월,16:45,쉬는시간,부속시설,복합부위,물리적힘 노출,기타,건물(문/창문/바닥/벽 등)
4,E0001913,고등학교,경기,안산교육지원청,공립,일반학생,여,2학년,2023-01-02,월,12:20,수업시간,교실,발,낙상,공부,가구(책상/의자/책장/탁자/침대 등)


### 함수 설정

In [ ]:
def count_to(df):
    series_copy = df.copy()
    series_copy.dropna(inplace=True)
    first_list = series_copy.tolist()

    words_to_split = [word for word in first_list if ',' in str(word)]
    for word in words_to_split:
        first_list.remove(word)
        first_list.extend(word.split(','))

    unique_words = list(set(first_list))
    counts = [first_list.count(word) for word in unique_words]

    count_df = pd.DataFrame({df.name: unique_words, '건수': counts})
    count_df = count_df.sort_values('건수', ascending=False).reset_index(drop=True)

    count_df.reset_index(drop=True, inplace=True)
    per = []

    for i in range(0, len(count_df)):
        per.append(round(count_df['건수'].loc[i] / count_df['건수'].sum() * 100, 2))
    count_df['퍼센트(%)'] = per

    count_df.sort_values(by='퍼센트(%)', ascending=False, inplace=True)
    count_df.reset_index(drop = True, inplace=True)

    return count_df

In [ ]:
def create_chart(dfs, theme):
    # 색상 팔레트
    palette = ["#5c7dd2","#92b8ff","#aeceff","#c7e4ff",'#c3b7eb', '#9590e6', '#837ed5']

    # 그래프 생성
    fig = px.bar(dfs, 
                 x=theme, 
                 y='퍼센트(%)', 
                 color=theme,  # theme을 색상으로 사용
                 orientation='v',  # 세로 막대 그래프 설정
                 text='퍼센트(%)',
                 color_discrete_sequence=palette,  # 사용자 지정 색상 팔레트 사용
                 custom_data=['건수', '퍼센트(%)'])

    # 레이아웃 설정
    fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체_Pro',
                        'color':'black'}), #전체 폰트 설정(로컬 폰트 사용 가능)
                hoverlabel=dict(        #호버 박스 폰트 설정
                    font_size=15,
                    font_family="KoPubWorld돋움체_Pro"
                ),
                paper_bgcolor='white',  # 전체 배경색
                plot_bgcolor='white',    # 플롯 배경색
                )

    # 텍스트 설정
    fig.update_traces(texttemplate='%{text:.1f}%', textposition='inside', textfont_size=12)

    # hover 시 정보 설정
    fig.update_traces(
        hovertemplate='<b><br>%{x}</b></br>%{customdata[1]}</br>건수: %{customdata[0]} 명<br><extra></extra>'
    )

    fig.show()

In [ ]:
def create_pie_chart(dfs, theme):
    palette = ["#5c7dd2","#92b8ff","#aeceff","#c7e4ff",'#c3b7eb', '#9590e6', '#837ed5']

    # 원 그래프 생성
    fig = px.pie(dfs, 
                 names=theme, 
                 values='퍼센트(%)', 
                 color=theme,  # theme을 색상으로 사용
                 color_discrete_sequence=palette,  # 사용자 지정 색상 팔레트 사용
                 custom_data=['건수', '퍼센트(%)'])

    # 레이아웃 설정
    fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체_Pro',
                        'color':'black'}), # 전체 폰트 설정(로컬 폰트 사용 가능)
                hoverlabel=dict(        # 호버 박스 폰트 설정
                    font_size=15,
                    font_family="KoPubWorld돋움체_Pro"
                ),
                paper_bgcolor='white',  # 전체 배경색
                plot_bgcolor='white',   # 플롯 배경색
                )

    # 텍스트 설정
    fig.update_traces(textinfo='percent+label', textfont_size=12)

    # hover 시 정보 설정
    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>퍼센트: %{percent:.1%}<br>건수: %{customdata[0]} 명<br><extra></extra>'
    )
    
    fig.show()


## 1. 학교 안전사고 현황 EDA

### 1-1. 연도별 사고 현황

- 전체 건수
- 사고 시간, 장소, 형태 추이

#### 연도별 사고 건수

In [ ]:
# 연도별 사고 건수 집계
filtered_df['연도'] = filtered_df['사고발생일'].dt.year
yearly_counts = filtered_df['연도'].value_counts().sort_index()

fig = px.bar(yearly_counts, 
             x=yearly_counts.index, 
             y=yearly_counts.values, 
             labels={'x': '연도', 'y': '사고 건수'})

fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체 Medium',
                        'color':'black'}), #전체 폰트 설정(로컬 폰트 사용 가능)
                hoverlabel=dict(        #호버 박스 폰트 설정
                    font_size=15,
                    font_family="KoPubWorld돋움체 Medium"
                ),
                paper_bgcolor='white',  # 전체 배경색
                plot_bgcolor='white',    # 플롯 배경색
                yaxis=dict(tickformat=',')
)
fig.update_traces(
        hovertemplate='<b><br>%{x}년</b></br>건수: %{y}<br><extra></extra>'
    )

fig.show()

In [ ]:
# 월별 사고 건수

# 월별 사고 건수를 집계하는 함수 정의
def monthly_counts(df):
    df['사고발생일'] = pd.to_datetime(df['사고발생일'])
    df['월'] = df['사고발생일'].dt.to_period('M').dt.to_timestamp()
    return df['월'].value_counts().sort_index()

# 각 연도별 월별 사고 건수 집계
monthly_counts_2019 = monthly_counts(df2019)
monthly_counts_2020 = monthly_counts(df2020)
monthly_counts_2021 = monthly_counts(df2021)
monthly_counts_2022 = monthly_counts(df2022)
monthly_counts_2023 = monthly_counts(df2023)

# 각 연도별 Series를 DataFrame으로 변환
monthly_counts_2019_df = monthly_counts_2019.reset_index()
monthly_counts_2020_df = monthly_counts_2020.reset_index()
monthly_counts_2021_df = monthly_counts_2021.reset_index()
monthly_counts_2022_df = monthly_counts_2022.reset_index()
monthly_counts_2023_df = monthly_counts_2023.reset_index()

# 컬럼명 통일
monthly_counts_2019_df.columns = ['월', '사고 건수']
monthly_counts_2020_df.columns = ['월', '사고 건수']
monthly_counts_2021_df.columns = ['월', '사고 건수']
monthly_counts_2022_df.columns = ['월', '사고 건수']
monthly_counts_2023_df.columns = ['월', '사고 건수']

# 연도별 DataFrame에 연도 정보 추가
monthly_counts_2019_df['연도'] = 2019
monthly_counts_2020_df['연도'] = 2020
monthly_counts_2021_df['연도'] = 2021
monthly_counts_2022_df['연도'] = 2022
monthly_counts_2023_df['연도'] = 2023

# 모든 연도별 DataFrame을 결합
all_data_monthly = pd.concat([monthly_counts_2019_df, monthly_counts_2020_df, monthly_counts_2021_df, monthly_counts_2022_df, monthly_counts_2023_df])

palette = ["#92b8ff","#aeceff",'#c3b7eb', '#9590e6', '#837ed5']
color_discrete_map = {2019: palette[0], 2020: palette[1], 2021: palette[2], 2022: palette[3], 2023: palette[4]}

# 연도를 문자열로 변환 (color_discrete_map과 일치하도록)
all_data_monthly['연도'] = all_data_monthly['연도'].astype(str)

# color_discrete_map도 문자열 키로 변경
color_discrete_map = {str(k): v for k, v in color_discrete_map.items()}

# 그래프 생성
fig = px.bar(all_data_monthly, 
             x='월', 
             y='사고 건수', 
             color='연도', 
             labels={'연도-월': '연도-월', '사고 건수': '사고 건수', '연도': '연도'}, 
             color_discrete_map=color_discrete_map)

fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체 Medium',
                           'color':'black'}), 
                hoverlabel=dict(
                    font_size=15,
                    font_family="KoPubWorld돋움체 Medium"
                ),
                paper_bgcolor='white',
                plot_bgcolor='white',
                yaxis=dict(tickformat=','))

fig.update_traces(
        hovertemplate='<b><br>%{x}</b></br>건수: %{y}<br><extra></extra>'
    )

# 그래프 표시
fig.show()

In [ ]:
def monthly_counts(df):
    df['사고발생일'] = pd.to_datetime(df['사고발생일'])
    df['월'] = df['사고발생일'].dt.month
    monthly_counts = df.groupby('월').size()
    return monthly_counts

# 각 연도별 월별 사고 건수 집계
monthly_counts_2019 = monthly_counts(df2019).reset_index(name='2019년')
monthly_counts_2020 = monthly_counts(df2020).reset_index(name='2020년')
monthly_counts_2021 = monthly_counts(df2021).reset_index(name='2021년')
monthly_counts_2022 = monthly_counts(df2022).reset_index(name='2022년')
monthly_counts_2023 = monthly_counts(df2023).reset_index(name='2023년')

# 모든 연도별 데이터를 하나의 데이터프레임으로 병합
merged_df = pd.merge(monthly_counts_2019, monthly_counts_2020, on='월', how='outer')
merged_df = pd.merge(merged_df, monthly_counts_2021, on='월', how='outer')
merged_df = pd.merge(merged_df, monthly_counts_2022, on='월', how='outer')
merged_df = pd.merge(merged_df, monthly_counts_2023, on='월', how='outer')

# 월을 인덱스로 설정
merged_df = merged_df.set_index('월')
merged_df.index = merged_df.index.map(lambda x: f'{x}월')

# NaN 값을 0으로 채움 (사고 건수가 없는 월)
merged_df = merged_df.fillna(0)

# 사용자 정의 색상 팔레트
colorscale = [
    [0, '#F7FBFC'],
    [0.2, '#E6F3F9'],
    [0.4, '#D6E6F2'],
    [0.6, '#B9D7EA'],
    [0.8, '#A2C5E2'],
    [1, '#769FCD']
]

# Plotly의 heatmap을 사용하여 시각화
fig = go.Figure(data=go.Heatmap(
                   z=merged_df.values,
                   x=merged_df.columns,
                   y=merged_df.index[::-1],
                   colorscale=colorscale,
                   text=merged_df.values,
                   texttemplate="%{z:,.0f}",
                   hovertemplate='<b>%{x} %{y}</b><br>사고 건수: %{z:,.0f} 건<extra></extra>',
                   ))

fig.update_layout(
    title='연도별 월별 사고 건수',
    font=dict(family="KoPubWorld돋움체_Pro", size=12)
)

fig.show()


#### 2019년

In [ ]:
df2019['월'] = df2019['사고발생일'].dt.month
yearly_counts = df2019['월'].value_counts().sort_index()

fig = px.bar(yearly_counts, 
             x=yearly_counts.index, 
             y=yearly_counts.values, 
             labels={'x': '월', 'y': '사고 건수'})

fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체_Pro',
                        'color':'black'}), #전체 폰트 설정(로컬 폰트 사용 가능)
                hoverlabel=dict(        #호버 박스 폰트 설정
                    font_size=15,
                    font_family="KoPubWorld돋움체_Pro"
                ),
                paper_bgcolor='white',  # 전체 배경색
                plot_bgcolor='white',    # 플롯 배경색
                yaxis=dict(tickformat=',',  title='사고 건수')
)
fig.update_traces(
        hovertemplate='<b><br>%{x}월</b></br>건수: %{y}<br><extra></extra>'
    )

fig.show()

In [ ]:
# 2019 사고자구분
df2019_1 = count_to(df2019['사고자구분'])
df2019_1

,사고자구분,건수,퍼센트(%)
0,일반학생,134616,96.87
1,체육특기학생,2548,1.83
2,특수학교(학급)학생,1381,0.99
3,교육활동참여자,283,0.20
4,교직원,85,0.06
5,교직원(계약직),36,0.03
6,교직원(일반직),6,0.00
7,일반인,4,0.00
8,교직원(교원),3,0.00


In [ ]:
create_chart(df2019_1, '사고자구분')


In [ ]:
create_pie_chart(df2019_1, '사고자구분')

In [ ]:
# 2019 사고자성별
df2019_2 = count_to(df2019['사고자성별'])
df2019_2

,사고자성별,건수,퍼센트(%)
0,남,93273,67.12
1,여,45690,32.88


In [ ]:
create_chart(df2019_2, '사고자성별')

In [ ]:
create_pie_chart(df2019_2, '사고자성별')

In [ ]:
# 2019 사고시간
df2019_3 = count_to(df2019['사고시간'])
df2019_3

,사고시간,건수,퍼센트(%)
0,체육수업,43588,31.37
1,점심시간,28993,20.86
2,수업시간,21800,15.69
3,휴식시간 및 청소시간,20626,14.84
4,학교행사,9677,6.96
5,등하교,6752,4.86
6,특별활동,6068,4.37
7,석식시간,811,0.58
8,기숙사생활,604,0.43
9,기타,43,0.03


In [ ]:
create_chart(df2019_3, '사고시간')

In [ ]:
create_pie_chart(df2019_3, '사고시간')

In [ ]:
# 2019 사고장소
df2019_4 = count_to(df2019['사고장소'])
df2019_4

,사고장소,건수,퍼센트(%)
0,운동장,50750,36.52
1,부속시설,37260,26.81
2,교실,24471,17.61
3,통로,20700,14.90
4,교외활동,5781,4.16
5,교외,1,0.00


In [ ]:
create_chart(df2019_4, '사고장소')

In [ ]:
create_pie_chart(df2019_4, '사고장소')

In [ ]:
# 2019 사고부위
df2019_5 = count_to(df2019['사고부위'])
df2019_5

,사고부위,건수,퍼센트(%)
0,손,33034,23.77
1,발,32330,23.27
2,머리(두부),27336,19.67
3,다리,17858,12.85
4,팔,13184,9.49
5,치아(구강),10843,7.80
6,흉복부,2438,1.75
7,기타,1897,1.37
8,복합부위,43,0.03


In [ ]:
create_chart(df2019_5, '사고부위')

In [ ]:
create_pie_chart(df2019_5, '사고부위')

In [ ]:
# 2019 사고형태
df2019_6 = count_to(df2019['사고형태'])
df2019_6

,사고형태,건수,퍼센트(%)
0,물리적힘 노출,54884,39.50
1,낙상-넘어짐,36162,26.02
2,사람과의 충돌,19355,13.93
3,낙상-미끄러짐,11906,8.57
4,기타,11232,8.08
5,낙상-떨어짐,5423,3.90
6,낙상,1,0.00


In [ ]:
create_chart(df2019_6, '사고형태')

In [ ]:
create_pie_chart(df2019_6, '사고형태')

In [ ]:
# 2019 사고당시활동
df2019_7 = count_to(df2019['사고당시활동'])
df2019_7

,사고당시활동,건수,퍼센트(%)
0,구기운동,56693,40.80
1,보행/주행,23893,17.19
2,장난/놀이,20135,14.49
3,기타운동,16006,11.52
4,기타,8281,5.96
5,식사/수면/휴식,7463,5.37
6,실험실습,3264,2.35
7,공부,3228,2.32


In [ ]:
create_chart(df2019_7, '사고당시활동')

In [ ]:
create_pie_chart(df2019_7, '사고당시활동')

In [ ]:
df2019['사고매개물'].unique()

array(['건물(문/창문/바닥/벽 등)', '자연(사람/동물/식물 등)', '가구(책상/의자/책장/탁자/침대 등)',
       '기계 도구류(기계선반, 재봉틀기계 등)', '운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등)', '기타',
       '날카로운 물건(칼/가위/송곳 등)', '열(불/뜨거운 물 등)', '운송용구(차/자전거/선박/항공기 등)'],
      dtype=object)

In [ ]:
# 2019 사고매개물
df2019_8 = count_to(df2019['사고매개물'].str.split('(').str[0]) # 괄호 내용 삭제
df2019_8

,사고매개물,건수,퍼센트(%)
0,운동,44465,32.00
1,건물,29372,21.14
2,자연,27078,19.49
3,기타,23645,17.02
4,가구,8501,6.12
5,날카로운 물건,2843,2.05
6,열,1480,1.07
7,기계 도구류,1014,0.73
8,운송용구,565,0.41


In [ ]:
create_chart(df2019_8, '사고매개물')

In [ ]:
create_pie_chart(df2019_8, '사고매개물')

#### 2020년

In [ ]:
df2020['월'] = df2020['사고발생일'].dt.month
yearly_counts = df2020['월'].value_counts().sort_index()

fig = px.bar(yearly_counts, 
             x=yearly_counts.index, 
             y=yearly_counts.values, 
             labels={'x': '월', 'y': '사고 건수'})

fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체_Pro',
                        'color':'black'}), #전체 폰트 설정(로컬 폰트 사용 가능)
                hoverlabel=dict(        #호버 박스 폰트 설정
                    font_size=15,
                    font_family="KoPubWorld돋움체_Pro"
                ),
                paper_bgcolor='white',  # 전체 배경색
                plot_bgcolor='white',    # 플롯 배경색
                yaxis=dict(tickformat=',',  title='사고 건수')
)
fig.update_traces(
        hovertemplate='<b><br>%{x}월</b></br>건수: %{y}<br><extra></extra>'
    )

fig.show()

In [ ]:
# 2020 사고자구분
df2020_1 = count_to(df2020['사고자구분'])
df2020_1

,사고자구분,건수,퍼센트(%)
0,일반학생,37315,94.58
1,체육특기학생,1496,3.79
2,특수학교(학급)학생,530,1.34
3,교육활동참여자,69,0.17
4,교직원(교원),22,0.06
5,교직원(일반직),14,0.04
6,교직원(계약직),9,0.02


In [ ]:
create_chart(df2020_1, '사고자구분')


In [ ]:
create_pie_chart(df2020_1, '사고자구분')

In [ ]:
# 2020 사고자성별
df2020_2 = count_to(df2020['사고자성별'])
df2020_2

,사고자성별,건수,퍼센트(%)
0,남,26347,66.78
1,여,13108,33.22


In [ ]:
create_chart(df2020_2, '사고자성별')

In [ ]:
create_pie_chart(df2020_2, '사고자성별')

In [ ]:
# 2020 사고시간
df2020_3 = count_to(df2020['사고시간'])
df2020_3

,사고시간,건수,퍼센트(%)
0,체육수업,13798,34.97
1,수업시간,7782,19.72
2,점심시간,5857,14.84
3,휴식시간 및 청소시간,4478,11.35
4,등하교,2536,6.43
5,특별활동,2067,5.24
6,기타,1527,3.87
7,학교행사,842,2.13
8,기숙사생활,335,0.85
9,석식시간,212,0.54


In [ ]:
create_chart(df2020_3, '사고시간')

In [ ]:
create_pie_chart(df2020_3, '사고시간')

In [ ]:
# 2020 사고장소
df2020_4 = count_to(df2020['사고장소'])
df2020_4

,사고장소,건수,퍼센트(%)
0,부속시설,13327,33.78
1,운동장,11850,30.03
2,교실,7802,19.77
3,통로,5087,12.89
4,교외활동,1388,3.52
5,교외,1,0.00


In [ ]:
create_chart(df2020_4, '사고장소')

In [ ]:
create_pie_chart(df2020_4, '사고장소')

In [ ]:
# 2020 사고부위
df2020_5 = count_to(df2020['사고부위'])
df2020_5

,사고부위,건수,퍼센트(%)
0,발,10429,26.43
1,손,9416,23.87
2,머리(두부),6315,16.01
3,다리,3580,9.07
4,복합부위,3482,8.83
5,팔,2597,6.58
6,치아(구강),1858,4.71
7,기타,1148,2.91
8,흉복부,630,1.60


In [ ]:
create_chart(df2020_5, '사고부위')

In [ ]:
create_pie_chart(df2020_5, '사고부위')

In [ ]:
# 2020 사고형태
df2020_6 = count_to(df2020['사고형태'])
df2020_6

,사고형태,건수,퍼센트(%)
0,물리적힘 노출,15252,38.66
1,낙상-넘어짐,10724,27.18
2,낙상-미끄러짐,6088,15.43
3,기타,4191,10.62
4,사람과의 충돌,1774,4.50
5,낙상-떨어짐,1407,3.57
6,낙상,19,0.05


In [ ]:
create_chart(df2020_6, '사고형태')

In [ ]:
create_pie_chart(df2020_6, '사고형태')

In [ ]:
# 2020 사고당시활동
df2020_7 = count_to(df2020['사고당시활동'])
df2020_7

,사고당시활동,건수,퍼센트(%)
0,구기운동,13950,35.36
1,보행/주행,6721,17.03
2,기타,6148,15.58
3,장난/놀이,5355,13.57
4,기타운동,2900,7.35
5,식사/수면/휴식,2143,5.43
6,공부,1133,2.87
7,실험실습,1105,2.80


In [ ]:
create_chart(df2020_7, '사고당시활동')

In [ ]:
create_pie_chart(df2020_7, '사고당시활동')

In [ ]:
df2020['사고매개물'].unique()

array(['기타', '자연(사람/동물/식물 등)', '건물(문/창문/바닥/벽 등)', '기계 도구류(기계선반, 재봉틀기계 등)',
       '가구(책상/의자/책장/탁자/침대 등)', '운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등)',
       '날카로운 물건(칼/가위/송곳 등)', '열(불/뜨거운 물 등)', '운송용구(차/자전거/선박/항공기 등)'],
      dtype=object)

In [ ]:
# 2020 사고매개물
df2020_8 = count_to(df2020['사고매개물'].str.split('(').str[0]) # 괄호 내용 삭제
df2020_8

,사고매개물,건수,퍼센트(%)
0,운동,12738,32.28
1,기타,7908,20.04
2,건물,7139,18.09
3,자연,6871,17.41
4,가구,3149,7.98
5,날카로운 물건,866,2.19
6,열,324,0.82
7,운송용구,250,0.63
8,기계 도구류,210,0.53


In [ ]:
create_chart(df2019_8, '사고매개물')

In [ ]:
create_pie_chart(df2019_8, '사고매개물')

#### 2021년

In [ ]:
df2021['월'] = df2021['사고발생일'].dt.month
yearly_counts = df2021['월'].value_counts().sort_index()

fig = px.bar(yearly_counts, 
             x=yearly_counts.index, 
             y=yearly_counts.values, 
             labels={'x': '월', 'y': '사고 건수'})

fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체_Pro',
                        'color':'black'}), #전체 폰트 설정(로컬 폰트 사용 가능)
                hoverlabel=dict(        #호버 박스 폰트 설정
                    font_size=15,
                    font_family="KoPubWorld돋움체_Pro"
                ),
                paper_bgcolor='white',  # 전체 배경색
                plot_bgcolor='white',    # 플롯 배경색
                yaxis=dict(tickformat=',',  title='사고 건수')
)
fig.update_traces(
        hovertemplate='<b><br>%{x}월</b></br>건수: %{y}<br><extra></extra>'
    )

fig.show()

In [ ]:
# 2021 사고자구분
df2021_1 = count_to(df2021['사고자구분'])
df2021_1

,사고자구분,건수,퍼센트(%)
0,일반학생,90982,96.12
1,체육특기학생,2391,2.53
2,특수학교(학급)학생,1050,1.11
3,교육활동참여자,103,0.11
4,교직원(교원),72,0.08
5,교직원(계약직),51,0.05
6,교직원(일반직),7,0.01
7,기타(일반인 등),3,0.00


In [ ]:
create_chart(df2021_1, '사고자구분')


In [ ]:
create_pie_chart(df2021_1, '사고자구분')

In [ ]:
# 2021 사고자성별
df2021_2 = count_to(df2021['사고자성별'])
df2021_2

,사고자성별,건수,퍼센트(%)
0,남,63118,66.68
1,여,31541,33.32


In [ ]:
create_chart(df2021_2, '사고자성별')

In [ ]:
create_pie_chart(df2021_2, '사고자성별')

In [ ]:
# 2021 사고시간
df2021_3 = count_to(df2021['사고시간'])
df2021_3

,사고시간,건수,퍼센트(%)
0,체육수업,37813,39.95
1,점심시간,14621,15.45
2,수업시간,14329,15.14
3,휴식시간 및 청소시간,10339,10.92
4,등하교,5494,5.80
5,특별활동,4094,4.32
6,기타,4081,4.31
7,학교행사,2893,3.06
8,기숙사생활,472,0.50
9,석식시간,424,0.45


In [ ]:
create_chart(df2021_3, '사고시간')

In [ ]:
create_pie_chart(df2021_3, '사고시간')

In [ ]:
# 2021 사고장소
df2021_4 = count_to(df2021['사고장소'])
df2021_4

,사고장소,건수,퍼센트(%)
0,부속시설,34130,36.06
1,운동장,29450,31.11
2,교실,15624,16.51
3,통로,12412,13.11
4,교외활동,3034,3.21
5,교외,9,0.01


In [ ]:
create_chart(df2021_4, '사고장소')

In [ ]:
create_pie_chart(df2021_4, '사고장소')

In [ ]:
# 2021 사고부위
df2021_5 = count_to(df2021['사고부위'])
df2021_5

,사고부위,건수,퍼센트(%)
0,손,25614,27.06
1,발,25336,26.77
2,머리(두부),13816,14.60
3,복합부위,10483,11.07
4,다리,7120,7.52
5,팔,5342,5.64
6,치아(구강),2899,3.06
7,기타,2622,2.77
8,흉복부,1427,1.51


In [ ]:
create_chart(df2021_5, '사고부위')

In [ ]:
create_pie_chart(df2021_5, '사고부위')

In [ ]:
# 2021 사고형태
df2021_6 = count_to(df2021['사고형태'])
df2021_6

,사고형태,건수,퍼센트(%)
0,물리적힘 노출,39251,41.47
1,낙상-넘어짐,25517,26.96
2,낙상-미끄러짐,15136,15.99
3,기타,8587,9.07
4,낙상-떨어짐,3071,3.24
5,사람과의 충돌,3016,3.19
6,낙상,81,0.09


In [ ]:
create_chart(df2021_6, '사고형태')

In [ ]:
create_pie_chart(df2021_6, '사고형태')

In [ ]:
# 2021 사고당시활동
df2021_7 = count_to(df2021['사고당시활동'])
df2021_7

,사고당시활동,건수,퍼센트(%)
0,구기운동,37308,39.41
1,기타,15529,16.41
2,보행/주행,15464,16.34
3,장난/놀이,11738,12.40
4,기타운동,6054,6.40
5,식사/수면/휴식,4193,4.43
6,공부,2295,2.42
7,실험실습,2078,2.20


In [ ]:
create_chart(df2021_7, '사고당시활동')

In [ ]:
create_pie_chart(df2021_7, '사고당시활동')

In [ ]:
df2021['사고매개물'].unique()

array(['건물(문/창문/바닥/벽 등)', '자연(사람/동물/식물 등)', '가구(책상/의자/책장/탁자/침대 등)',
       '운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등)', '운송용구(차/자전거/선박/항공기 등)', '기타',
       '기계 도구류(기계선반, 재봉틀기계 등)', '열(불/뜨거운 물 등)', '날카로운 물건(칼/가위/송곳 등)'],
      dtype=object)

In [ ]:
# 2021 사고매개물
df2021_8 = count_to(df2021['사고매개물'].str.split('(').str[0]) # 괄호 내용 삭제
df2021_8

,사고매개물,건수,퍼센트(%)
0,운동,34369,36.31
1,자연,17620,18.61
2,기타,17306,18.28
3,건물,16159,17.07
4,가구,6077,6.42
5,날카로운 물건,1827,1.93
6,열,608,0.64
7,운송용구,439,0.46
8,기계 도구류,254,0.27


In [ ]:
create_chart(df2021_8, '사고매개물')

In [ ]:
create_pie_chart(df2021_8, '사고매개물')

#### 2022년

In [ ]:
df2019['월'] = df2019['사고발생일'].dt.month
yearly_counts = df2019['월'].value_counts().sort_index()

fig = px.bar(yearly_counts, 
             x=yearly_counts.index, 
             y=yearly_counts.values, 
             labels={'x': '월', 'y': '사고 건수'})

fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체_Pro',
                        'color':'black'}), #전체 폰트 설정(로컬 폰트 사용 가능)
                hoverlabel=dict(        #호버 박스 폰트 설정
                    font_size=15,
                    font_family="KoPubWorld돋움체_Pro"
                ),
                paper_bgcolor='white',  # 전체 배경색
                plot_bgcolor='white',    # 플롯 배경색
                yaxis=dict(tickformat=',',  title='사고 건수')
)
fig.update_traces(
        hovertemplate='<b><br>%{x}월</b></br>건수: %{y}<br><extra></extra>'
    )

fig.show()

In [ ]:
# 2019 사고자구분
df2019_1 = count_to(df2019['사고자구분'])
df2019_1

,사고자구분,건수,퍼센트(%)
0,일반학생,134616,96.87
1,체육특기학생,2548,1.83
2,특수학교(학급)학생,1381,0.99
3,교육활동참여자,283,0.20
4,교직원,85,0.06
5,교직원(계약직),36,0.03
6,교직원(일반직),6,0.00
7,일반인,4,0.00
8,교직원(교원),3,0.00


In [ ]:
create_chart(df2019_1, '사고자구분')


In [ ]:
create_pie_chart(df2019_1, '사고자구분')

In [ ]:
# 2019 사고자성별
df2019_2 = count_to(df2019['사고자성별'])
df2019_2

,사고자성별,건수,퍼센트(%)
0,남,93273,67.12
1,여,45690,32.88


In [ ]:
create_chart(df2019_2, '사고자성별')

In [ ]:
create_pie_chart(df2019_2, '사고자성별')

In [ ]:
# 2019 사고시간
df2019_3 = count_to(df2019['사고시간'])
df2019_3

,사고시간,건수,퍼센트(%)
0,체육수업,43588,31.37
1,점심시간,28993,20.86
2,수업시간,21800,15.69
3,휴식시간 및 청소시간,20626,14.84
4,학교행사,9677,6.96
5,등하교,6752,4.86
6,특별활동,6068,4.37
7,석식시간,811,0.58
8,기숙사생활,604,0.43
9,기타,43,0.03


In [ ]:
create_chart(df2019_3, '사고시간')

In [ ]:
create_pie_chart(df2019_3, '사고시간')

In [ ]:
# 2019 사고장소
df2019_4 = count_to(df2019['사고장소'])
df2019_4

,사고장소,건수,퍼센트(%)
0,운동장,50750,36.52
1,부속시설,37260,26.81
2,교실,24471,17.61
3,통로,20700,14.90
4,교외활동,5781,4.16
5,교외,1,0.00


In [ ]:
create_chart(df2019_4, '사고장소')

In [ ]:
create_pie_chart(df2019_4, '사고장소')

In [ ]:
# 2019 사고부위
df2019_5 = count_to(df2019['사고부위'])
df2019_5

,사고부위,건수,퍼센트(%)
0,손,33034,23.77
1,발,32330,23.27
2,머리(두부),27336,19.67
3,다리,17858,12.85
4,팔,13184,9.49
5,치아(구강),10843,7.80
6,흉복부,2438,1.75
7,기타,1897,1.37
8,복합부위,43,0.03


In [ ]:
create_chart(df2019_5, '사고부위')

In [ ]:
create_pie_chart(df2019_5, '사고부위')

In [ ]:
# 2019 사고형태
df2019_6 = count_to(df2019['사고형태'])
df2019_6

,사고형태,건수,퍼센트(%)
0,물리적힘 노출,54884,39.50
1,낙상-넘어짐,36162,26.02
2,사람과의 충돌,19355,13.93
3,낙상-미끄러짐,11906,8.57
4,기타,11232,8.08
5,낙상-떨어짐,5423,3.90
6,낙상,1,0.00


In [ ]:
create_chart(df2019_6, '사고형태')

In [ ]:
create_pie_chart(df2019_6, '사고형태')

In [ ]:
# 2019 사고당시활동
df2019_7 = count_to(df2019['사고당시활동'])
df2019_7

,사고당시활동,건수,퍼센트(%)
0,구기운동,56693,40.80
1,보행/주행,23893,17.19
2,장난/놀이,20135,14.49
3,기타운동,16006,11.52
4,기타,8281,5.96
5,식사/수면/휴식,7463,5.37
6,실험실습,3264,2.35
7,공부,3228,2.32


In [ ]:
create_chart(df2019_7, '사고당시활동')

In [ ]:
create_pie_chart(df2019_7, '사고당시활동')

In [ ]:
df2019['사고매개물'].unique()

array(['건물(문/창문/바닥/벽 등)', '자연(사람/동물/식물 등)', '가구(책상/의자/책장/탁자/침대 등)',
       '기계 도구류(기계선반, 재봉틀기계 등)', '운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등)', '기타',
       '날카로운 물건(칼/가위/송곳 등)', '열(불/뜨거운 물 등)', '운송용구(차/자전거/선박/항공기 등)'],
      dtype=object)

In [ ]:
# 2019 사고매개물
df2019_8 = count_to(df2019['사고매개물'].str.split('(').str[0]) # 괄호 내용 삭제
df2019_8

,사고매개물,건수,퍼센트(%)
0,운동,44465,32.00
1,건물,29372,21.14
2,자연,27078,19.49
3,기타,23645,17.02
4,가구,8501,6.12
5,날카로운 물건,2843,2.05
6,열,1480,1.07
7,기계 도구류,1014,0.73
8,운송용구,565,0.41


In [ ]:
create_chart(df2019_8, '사고매개물')

In [ ]:
create_pie_chart(df2019_8, '사고매개물')

#### 2023년

In [ ]:
df2019['월'] = df2019['사고발생일'].dt.month
yearly_counts = df2019['월'].value_counts().sort_index()

fig = px.bar(yearly_counts, 
             x=yearly_counts.index, 
             y=yearly_counts.values, 
             labels={'x': '월', 'y': '사고 건수'})

fig.update_layout(
                font=dict({'family':'KoPubWorld돋움체_Pro',
                        'color':'black'}), #전체 폰트 설정(로컬 폰트 사용 가능)
                hoverlabel=dict(        #호버 박스 폰트 설정
                    font_size=15,
                    font_family="KoPubWorld돋움체_Pro"
                ),
                paper_bgcolor='white',  # 전체 배경색
                plot_bgcolor='white',    # 플롯 배경색
                yaxis=dict(tickformat=',',  title='사고 건수')
)
fig.update_traces(
        hovertemplate='<b><br>%{x}월</b></br>건수: %{y}<br><extra></extra>'
    )

fig.show()

In [ ]:
# 2019 사고자구분
df2019_1 = count_to(df2019['사고자구분'])
df2019_1

,사고자구분,건수,퍼센트(%)
0,일반학생,134616,96.87
1,체육특기학생,2548,1.83
2,특수학교(학급)학생,1381,0.99
3,교육활동참여자,283,0.20
4,교직원,85,0.06
5,교직원(계약직),36,0.03
6,교직원(일반직),6,0.00
7,일반인,4,0.00
8,교직원(교원),3,0.00


In [ ]:
create_chart(df2019_1, '사고자구분')


In [ ]:
create_pie_chart(df2019_1, '사고자구분')

In [ ]:
# 2019 사고자성별
df2019_2 = count_to(df2019['사고자성별'])
df2019_2

,사고자성별,건수,퍼센트(%)
0,남,93273,67.12
1,여,45690,32.88


In [ ]:
create_chart(df2019_2, '사고자성별')

In [ ]:
create_pie_chart(df2019_2, '사고자성별')

In [ ]:
# 2019 사고시간
df2019_3 = count_to(df2019['사고시간'])
df2019_3

,사고시간,건수,퍼센트(%)
0,체육수업,43588,31.37
1,점심시간,28993,20.86
2,수업시간,21800,15.69
3,휴식시간 및 청소시간,20626,14.84
4,학교행사,9677,6.96
5,등하교,6752,4.86
6,특별활동,6068,4.37
7,석식시간,811,0.58
8,기숙사생활,604,0.43
9,기타,43,0.03


In [ ]:
create_chart(df2019_3, '사고시간')

In [ ]:
create_pie_chart(df2019_3, '사고시간')

In [ ]:
# 2019 사고장소
df2019_4 = count_to(df2019['사고장소'])
df2019_4

,사고장소,건수,퍼센트(%)
0,운동장,50750,36.52
1,부속시설,37260,26.81
2,교실,24471,17.61
3,통로,20700,14.90
4,교외활동,5781,4.16
5,교외,1,0.00


In [ ]:
create_chart(df2019_4, '사고장소')

In [ ]:
create_pie_chart(df2019_4, '사고장소')

In [ ]:
# 2019 사고부위
df2019_5 = count_to(df2019['사고부위'])
df2019_5

,사고부위,건수,퍼센트(%)
0,손,33034,23.77
1,발,32330,23.27
2,머리(두부),27336,19.67
3,다리,17858,12.85
4,팔,13184,9.49
5,치아(구강),10843,7.80
6,흉복부,2438,1.75
7,기타,1897,1.37
8,복합부위,43,0.03


In [ ]:
create_chart(df2019_5, '사고부위')

In [ ]:
create_pie_chart(df2019_5, '사고부위')

In [ ]:
# 2019 사고형태
df2019_6 = count_to(df2019['사고형태'])
df2019_6

,사고형태,건수,퍼센트(%)
0,물리적힘 노출,54884,39.50
1,낙상-넘어짐,36162,26.02
2,사람과의 충돌,19355,13.93
3,낙상-미끄러짐,11906,8.57
4,기타,11232,8.08
5,낙상-떨어짐,5423,3.90
6,낙상,1,0.00


In [ ]:
create_chart(df2019_6, '사고형태')

In [ ]:
create_pie_chart(df2019_6, '사고형태')

In [ ]:
# 2019 사고당시활동
df2019_7 = count_to(df2019['사고당시활동'])
df2019_7

,사고당시활동,건수,퍼센트(%)
0,구기운동,56693,40.80
1,보행/주행,23893,17.19
2,장난/놀이,20135,14.49
3,기타운동,16006,11.52
4,기타,8281,5.96
5,식사/수면/휴식,7463,5.37
6,실험실습,3264,2.35
7,공부,3228,2.32


In [ ]:
create_chart(df2019_7, '사고당시활동')

In [ ]:
create_pie_chart(df2019_7, '사고당시활동')

In [ ]:
df2019['사고매개물'].unique()

array(['건물(문/창문/바닥/벽 등)', '자연(사람/동물/식물 등)', '가구(책상/의자/책장/탁자/침대 등)',
       '기계 도구류(기계선반, 재봉틀기계 등)', '운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등)', '기타',
       '날카로운 물건(칼/가위/송곳 등)', '열(불/뜨거운 물 등)', '운송용구(차/자전거/선박/항공기 등)'],
      dtype=object)

In [ ]:
# 2019 사고매개물
df2019_8 = count_to(df2019['사고매개물'].str.split('(').str[0]) # 괄호 내용 삭제
df2019_8

,사고매개물,건수,퍼센트(%)
0,운동,44465,32.00
1,건물,29372,21.14
2,자연,27078,19.49
3,기타,23645,17.02
4,가구,8501,6.12
5,날카로운 물건,2843,2.05
6,열,1480,1.07
7,기계 도구류,1014,0.73
8,운송용구,565,0.41


In [ ]:
create_chart(df2019_8, '사고매개물')

In [ ]:
create_pie_chart(df2019_8, '사고매개물')